## Introducción

En este notebook se realiza un análisis geográfico de los casos judiciales iniciados por los Ministerios Públicos en Argentina.

El objetivo es explorar diferencias territoriales en volumen de casos y composición penal, así como evaluar posibles señales de heterogeneidad institucional en la calidad del registro de información.

El análisis es descriptivo y exploratorio: no se interpretan los volúmenes como tasas de criminalidad ni se realizan inferencias causales.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (11, 5)

# Dataset EDA (incluye imputaciones controladas y flags)
DATA_FILE = Path("..") / "data" / "analytics" / "casos_geografico.csv"
df = pd.read_csv(DATA_FILE, low_memory=False)

# Asegurar fecha
df["caso_fecha_inicio"] = pd.to_datetime(df["caso_fecha_inicio"], errors="coerce")

df.shape

(6686833, 20)

## Preparación mínima

Se asegura el nivel de agregación por caso (`caso_id`) y se generan columnas auxiliares si es necesario.


In [ ]:
# Filtro mínimo: casos con provincia
df = df[df["provincia_nombre"].notna()].copy()

# Año (útil para cortes si hace falta)
df["anio_inicio"] = df["caso_fecha_inicio"].dt.year

df[["provincia_nombre", "caso_id", "anio_inicio"]].head()


## Ranking de provincias por cantidad de casos

Se calcula el número de casos únicos por provincia y su participación porcentual sobre el total.


In [ ]:
casos_por_provincia = (
    df.groupby("provincia_nombre")["caso_id"]
    .nunique()
    .sort_values(ascending=False)
)

ranking_provincias = casos_por_provincia.reset_index(name="casos")
ranking_provincias["participacion_pct"] = ranking_provincias["casos"] / ranking_provincias["casos"].sum() * 100

ranking_provincias.head(10)

# Top 15 para visualización
top15 = ranking_provincias.head(15)

sns.barplot(data=top15, x="casos", y="provincia_nombre", orient="h")
plt.title("Top 15 provincias por cantidad de casos (casos únicos)")
plt.xlabel("Cantidad de casos")
plt.ylabel("Provincia")
plt.show()

# Concentración: cuánto aportan las top N
def concentracion_top_n(ranking_df, n):
    return ranking_df.head(n)["participacion_pct"].sum()

for n in [3, 5, 10]:
    print(f"Participación acumulada Top {n}: {concentracion_top_n(ranking_provincias, n):.2f}%")
# Interpretación sugerida: si el top 5 o top 10 concentra un porcentaje muy alto, existe concentración territorial del volumen de causas iniciadas.



## Perfil penal por provincia

Se compara la composición de delitos entre provincias (en términos de proporciones) para evaluar si existen perfiles penales diferenciados.

Se trabaja con un subconjunto para mantener legibilidad:
- Top 8 provincias por volumen
- Top 10 delitos globales


In [ ]:
# Top provincias por volumen
top_provincias = ranking_provincias.head(8)["provincia_nombre"].tolist()

# Top delitos globales (excluye "No informado")
top_delitos = (
    df[df["delito_descripcion"] != "No informado"]
    .groupby("delito_descripcion")["caso_id"]
    .nunique()
    .sort_values(ascending=False)
    .head(10)
    .index
    .tolist()
)

df_sub = df[
    df["provincia_nombre"].isin(top_provincias) &
    df["delito_descripcion"].isin(top_delitos)
].copy()

df_sub.shape

# Conteo por provincia-delito (casos únicos)
prov_delito = (
    df_sub.groupby(["provincia_nombre", "delito_descripcion"])["caso_id"]
    .nunique()
    .reset_index(name="casos")
)

# Normalizar por provincia (proporciones)
prov_tot = prov_delito.groupby("provincia_nombre")["casos"].sum().reset_index(name="total")

prov_delito = prov_delito.merge(prov_tot, on="provincia_nombre", how="left")
prov_delito["share_pct"] = prov_delito["casos"] / prov_delito["total"] * 100

prov_delito.head()

# Heatmap (proporciones)
pivot = prov_delito.pivot(index="provincia_nombre", columns="delito_descripcion", values="share_pct").fillna(0)

plt.figure(figsize=(14, 6))
sns.heatmap(pivot, annot=False, cmap="Blues")
plt.title("Perfil penal por provincia (Top 8 provincias x Top 10 delitos) – % dentro de provincia")
plt.xlabel("Delito")
plt.ylabel("Provincia")
plt.show()

# Lectura recomendada del heatmap: no se comparan valores absolutos, sino **composición** dentro de cada provincia. Esto evita interpretaciones erróneas por tamaño poblacional.


## Comparación entre circunscripciones (dentro de una provincia)

Se evalúa la distribución de casos dentro de una provincia en sus circunscripciones.
Para evitar ruido, se realiza sobre la provincia con mayor volumen (top 1), pero puede replicarse para otras.


In [ ]:
prov_obj = ranking_provincias.iloc[0]["provincia_nombre"]
prov_obj

df_prov = df[df["provincia_nombre"] == prov_obj].copy()

# Chequear si existen circunscripciones
df_prov[["circunscripcion_descripcion", "circunscripcion_id"]].isna().mean() * 100

# Ranking de circunscripciones por casos dentro de la provincia
circ_rank = (
    df_prov.dropna(subset=["circunscripcion_descripcion"])
    .groupby("circunscripcion_descripcion")["caso_id"]
    .nunique()
    .sort_values(ascending=False)
    .reset_index(name="casos")
)

circ_rank.head(15)

